In [15]:
import os
import sys
import os
os.chdir('/scratch/nhl256/dl_project/code/')
from PIL import Image

import random

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from data_helper import *
from nhung_data_helper import FastRCNNLabeledDataset
from helper import *


import math
import pickle
import time
import copy
import utils

### 05/08/20

- Concat 6 images along 1 dim before passing through the model
- Saving the feature extractor and the fasterRCNN model
- ResNet18 for feature extraction using the weights from self-supervised task ('/scratch/nhl256/dl_project/model/pretrain_colorization_resnet18.pth')
- optimizer = torch.optim.Adam(params, lr=0.0001)


In [2]:
# All the images are saved in image_folder
# All the labels are saved in the annotation_csv file

# image_folder = '../data'
# annotation_csv = '../data/annotation.csv'

image_folder = 'data/data'
annotation_csv = 'data/data/annotation.csv'


cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
print(device)

# image_folder = '/Users/nhungle/Downloads/dl20_data'
# annotation_csv = '/Users/nhungle/Downloads/dl20_data/annotation.csv'

cuda:0


In [3]:
print(cuda)

True


In [4]:
labeled_scene_index = np.arange(106, 134)
random.seed(1008)
random.shuffle(labeled_scene_index)
train_labeled_scene_index = labeled_scene_index[:22]
val_labeled_scene_index = labeled_scene_index[22:26]
test_labeled_scene_index = labeled_scene_index[26:]
BATCH_SIZE=1


transform = torchvision.transforms.ToTensor()
fasterRCNN_trainset = FastRCNNLabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=train_labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
train_loader = torch.utils.data.DataLoader(fasterRCNN_trainset,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=2, collate_fn=collate_fn)


fasterRCNN_valset = FastRCNNLabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=val_labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
val_loader = torch.utils.data.DataLoader(fasterRCNN_valset,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=2, collate_fn=collate_fn)


fasterRCNN_testset = FastRCNNLabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=test_labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
test_loader = torch.utils.data.DataLoader(fasterRCNN_testset,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=2, collate_fn=collate_fn)

In [5]:
train_loader.__len__()

2772

In [6]:
class color_encoder(nn.Module):
    def __init__(self):
        super(color_encoder, self).__init__()
        encoder = torchvision.models.resnet18(pretrained=False)
        encoder = list(encoder.children())[1:-3]
        encoder = [nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)] + encoder
        self.encoder = nn.Sequential(*encoder)
        
    def forward(self, x):
        x = self.encoder(x)
        return x 
    
class _SameDecoder(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, num_layers, out_activation = 'relu'):
        super(_SameDecoder, self).__init__()
        layers = [
            nn.ConvTranspose2d(in_channels, in_channels, kernel_size=kernel_size, stride=stride),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        ]
        layers += [
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True) 
        ]*(num_layers-2)
        
        layers += [
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True) if out_activation == 'relu' else nn.Sigmoid(),
        ]
        self.decode = nn.Sequential(*layers)
    def forward(self, x):
        return self.decode(x)
    
def get_upsampling_weight(in_channels, out_channels, kernel_size):
    """Make a 2D bilinear kernel suitable for upsampling"""
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:kernel_size, :kernel_size]
    filt = (1 - abs(og[0] - center) / factor) * \
           (1 - abs(og[1] - center) / factor)
    weight = np.zeros((in_channels, out_channels, kernel_size, kernel_size),
                      dtype=np.float64)
    weight[range(in_channels), range(out_channels), :, :] = filt
    return torch.from_numpy(weight).float()

class color_decoder(nn.Module):
    def __init__(self):
        super(color_decoder, self).__init__()
        self.dec1 = _SameDecoder(256, 512,kernel_size=2, stride=2, num_layers=4)
        self.dec2 = _SameDecoder(512, 256,kernel_size=2, stride=2, num_layers=4)
        self.dec3 = _SameDecoder(256, 128,kernel_size=2, stride=2, num_layers=2)
        self.dec4 = _SameDecoder(128, 64,kernel_size=2, stride=2, num_layers=2)
        self.conv_out = nn.Conv2d(64, 2, 3 , padding=1)
        
        self.final_upsample = nn.Upsample((256, 306), mode='bilinear', align_corners=False)
        #self.sigmoid = nn.Sigmoid()
        self._initialize_weights()
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.ConvTranspose2d):
                assert m.kernel_size[0] == m.kernel_size[1]
                initial_weight = get_upsampling_weight(
                    m.in_channels, m.out_channels, m.kernel_size[0])
                m.weight.data.copy_(initial_weight)
                
    def forward(self, x):
        x = self.dec1(x)
        x= self.dec2(x)
        x = self.dec3(x)
        x = self.dec4(x)
        x = self.conv_out(x)
        #print('after layers the size is', x.size())
        x = self.final_upsample(x)
        #x = self.sigmoid(x)
        return x
    
class color_model(nn.Module):
    def __init__(self):
        super(color_model, self).__init__()
        self.encoder = color_encoder()
        self.decoder = color_decoder()
        
    def forward(self, x):
        mid = self.encoder(x)
        output = self.decoder(mid)
        
        return output

In [7]:
color_checkpoint = torch.load('/scratch/nhl256/dl_project/model/pretrain_colorization_resnet18.pth')
pretrain_model = color_model()
pretrain_model.load_state_dict(color_checkpoint['best_model'])

<All keys matched successfully>

### Get Feature Extractor

In [8]:
pretrain_encoder = pretrain_model.encoder.encoder
pretrain_encoder = list(pretrain_encoder.children())[1:]
feature_extractor = [nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)] + pretrain_encoder
feature_extractor = nn.Sequential(*feature_extractor)
feature_extractor.to(device)
for param in feature_extractor.parameters():
    param.requires_grad = True

In [9]:
def concat_features(features, dim = 2):
    #dim 0 ==> stacking the images in the channel dimension
    #dim 1 ==> stacking the images in row dimension
    #dim 2 ==> stacking the images in column dimension
    tensor_tuples = torch.unbind(features, dim=0)
    concatenated_fm = torch.cat(tensor_tuples, dim=dim)
    return concatenated_fm 

### Inspect if it works on a pretrained FasterRCNN

In [10]:
sample, targets = next(iter(train_loader))
sample = torch.stack(sample)
sample = sample.to(device)
batchsize = sample.shape[0]
fe_batch = []
for i in range(batchsize):
    image_tensor = sample[i]
    features = feature_extractor(image_tensor)
    #print(features.shape)
    features = concat_features(features) # torch.Size([256, 16, 120])
    features = features.view(3, 256, 40*16)
    #print(features.shape)
    fe_batch.append(features)

images = list(image.to(device) for image in fe_batch)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [11]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False,pretrained_backbone=False)
model = model.to(device)
output1 = model(images, targets)
output1

{'loss_classifier': tensor(17.8849, device='cuda:0', grad_fn=<NllLossBackward>),
 'loss_box_reg': tensor(0.2243, device='cuda:0', grad_fn=<DivBackward0>),
 'loss_objectness': tensor(1.8392, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(3.3491, device='cuda:0', grad_fn=<DivBackward0>)}

## Train One Epoch

In [26]:
import math
import sys
import time
import torch

import utils

In [16]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False,pretrained_backbone=False)
model = model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001)
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 1
epoch = 0
print_freq = 20

In [17]:
def train_one_epoch(feature_extractor, model, optimizer, data_loader, device, epoch, print_freq):
    feature_extractor.train()
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = utils.warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for sample, targets in metric_logger.log_every(data_loader, print_freq, header):
        sample = torch.stack(sample)
        sample = sample.to(device)
        batchsize = sample.shape[0]
        fe_batch = []
        for i in range(batchsize):
            image_tensor = sample[i]
            features = feature_extractor(image_tensor)
            #print(features.shape)
            features = concat_features(features)
            features = features.view(3, 256, 40*16)
            #print(features.shape)
            fe_batch.append(features)

        images = list(image.to(device) for image in fe_batch)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        #print(loss_dict)

        losses = sum(loss for loss in loss_dict.values())
        #print(losses)

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return losses

### Inspect train_one_epoch

In [18]:
train_one_epoch(feature_extractor, model, optimizer, test_loader, device, epoch, print_freq)

Epoch: [0]  [  0/252]  eta: 0:05:50  lr: 0.000000  loss: 20.4135 (20.4135)  loss_classifier: 15.3485 (15.3485)  loss_box_reg: 0.0171 (0.0171)  loss_objectness: 0.8709 (0.8709)  loss_rpn_box_reg: 4.1771 (4.1771)  time: 1.3890  data: 0.7828  max mem: 3011
Epoch: [0]  [ 20/252]  eta: 0:01:03  lr: 0.000008  loss: 4.1465 (7.8224)  loss_classifier: 0.3130 (3.7392)  loss_box_reg: 0.0375 (0.0368)  loss_objectness: 0.5850 (0.6020)  loss_rpn_box_reg: 3.1668 (3.4444)  time: 0.2167  data: 0.0043  max mem: 3450
Epoch: [0]  [ 40/252]  eta: 0:00:51  lr: 0.000016  loss: 1.4301 (4.7714)  loss_classifier: 0.1098 (1.9840)  loss_box_reg: 0.0143 (0.0292)  loss_objectness: 0.2926 (0.4689)  loss_rpn_box_reg: 1.0413 (2.2893)  time: 0.2149  data: 0.0038  max mem: 3450
Epoch: [0]  [ 60/252]  eta: 0:00:45  lr: 0.000024  loss: 1.1898 (3.5747)  loss_classifier: 0.1113 (1.3739)  loss_box_reg: 0.0052 (0.0217)  loss_objectness: 0.1867 (0.3788)  loss_rpn_box_reg: 0.8219 (1.8003)  time: 0.2159  data: 0.0041  max mem: 3

tensor(0.8828, device='cuda:0', grad_fn=<AddBackward0>)

In [19]:
sample, targets = next(iter(train_loader))
sample = torch.stack(sample)
sample = sample.to(device)
batchsize = sample.shape[0]
fe_batch = []
for i in range(batchsize):
    image_tensor = sample[i]
    features = feature_extractor(image_tensor)
    #print(features.shape)
    features = concat_features(features)
    features = features.view(3, 256, 40*16)
    #print(features.shape)
    fe_batch.append(features)

images = list(image.to(device) for image in fe_batch)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [20]:
model.eval()
predictions = model(images)
print(predictions)

[{'boxes': tensor([], device='cuda:0', size=(0, 4), grad_fn=<StackBackward>), 'labels': tensor([], device='cuda:0', dtype=torch.int64), 'scores': tensor([], device='cuda:0', grad_fn=<IndexBackward>)}]


In [ ]:
# model_test = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# model_test = model_test.to(device)
# model_test.eval()
# x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# x = list(image.to(device) for image in x)
# predictions = model_test(x)
# predictions

## Evaluate One Epoch

In [21]:
sample, targets = next(iter(val_loader))
sample = torch.stack(sample)
sample = sample.to(device)

In [22]:
def reorder_coord(pred_bboxes):
    xmin, ymin, xmax, ymax = pred_bboxes.unbind(1)
    return torch.stack((xmax, xmax, xmin, xmin, ymax, ymin, ymax, ymin), dim=1).view(-1, 2, 4)

def get_bounding_boxes(samples):
    # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
    # You need to return a tuple with size 'batch_size' and each element is a cuda tensor [N, 2, 4]
    # where N is the number of object

    #Preparing inputs
    batchsize = samples.shape[0]
    fe_batch = []
    for i in range(batchsize):
        image_tensor = sample[i]
        features = feature_extractor(image_tensor)
        #print(features.shape)
        features = concat_features(features)
        features = features.view(3, 256, 40*16)
        #print(features.shape)
        fe_batch.append(features)

    images = list(image.to(device) for image in fe_batch)
    predictions = model(images)
    res = []
    for i in range(len(predictions)):
        prediction = predictions[i]
        pred_bboxes = prediction['boxes']
        reorder_pred_bboxes = reorder_coord(pred_bboxes)
        res.append(reorder_pred_bboxes)

    return res

In [23]:
def eval_one_epoch(feature_extractor, model, dataloader):
    model.eval()
    feature_extractor.eval()
    total = 0
    total_ats_bounding_boxes = 0
    for i, data in enumerate(dataloader):
        total += 1
        sample, target = data
        sample = torch.stack(sample)
        sample = sample.cuda()

        predicted_bounding_boxes = get_bounding_boxes(sample)[0].cpu()
        

        ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes,
                                                        target[0]['bounding_box'])
#         print('Number of pred bboxes {}'.format(predicted_bounding_boxes.shape))
#         print('ats_bounding_boxes {}'.format(ats_bounding_boxes))

        total_ats_bounding_boxes += ats_bounding_boxes
    return total_ats_bounding_boxes

In [35]:
total_ats_bounding_boxes = eval_one_epoch(feature_extractor, model, test_loader)


In [36]:
total_ats_bounding_boxes

tensor(0.)

## Train and Eval for multiple epochs

In [24]:
# Get model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False,pretrained_backbone=False)
model = model.to(device)

# Get feature extractor
pretrain_encoder = pretrain_model.encoder.encoder
pretrain_encoder = list(pretrain_encoder.children())[1:]
feature_extractor = [nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)] + pretrain_encoder
feature_extractor = nn.Sequential(*feature_extractor)
feature_extractor.to(device)
for param in feature_extractor.parameters():
    param.requires_grad = True


# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001)
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10
epoch = 0
print_freq = 500

In [25]:
import pickle

In [26]:
def train_val(feature_extractor, model, train_loader, val_loader, num_epochs=10):
    best_model_wts = {'feature_extractor': copy.deepcopy(feature_extractor.state_dict()),
                       'fasterRCNN': copy.deepcopy(model.state_dict())
                                      }
    losses = []
    total_ats = []
    best_total_ats = -1
    
    for epoch in range(num_epochs):
        loss = train_one_epoch(feature_extractor, model, optimizer, train_loader, device, epoch, print_freq)
        lr_scheduler.step()
        total_ats_bounding_boxes = eval_one_epoch(feature_extractor, model, val_loader)
        losses.append(loss)
        total_ats.append(total_ats_bounding_boxes)
        print('epoch {} loss {} total_ats {}'.format(epoch, loss, total_ats))

        if total_ats_bounding_boxes > best_total_ats:
            best_total_ats = total_ats_bounding_boxes
            best_model_wts = {'feature_extractor': copy.deepcopy(feature_extractor.state_dict()),
                               'fasterRCNN': copy.deepcopy(model.state_dict())
                                     }
            torch.save(best_model_wts, '/scratch/nhl256/dl_project/model/object_detection_resnet18_0509_epoch{}.pth'.format(epoch))

    return losses, total_ats, best_model_wts

In [27]:
losses, total_ats, best_model_wts = train_val(feature_extractor, model,
                                              train_loader, val_loader,
                                              num_epochs=10)

Epoch: [0]  [   0/2772]  eta: 0:28:52  lr: 0.000000  loss: 13.8289 (13.8289)  loss_classifier: 10.7143 (10.7143)  loss_box_reg: 0.0666 (0.0666)  loss_objectness: 0.6826 (0.6826)  loss_rpn_box_reg: 2.3655 (2.3655)  time: 0.6250  data: 0.2597  max mem: 4431
Epoch: [0]  [ 500/2772]  eta: 0:08:12  lr: 0.000050  loss: 0.7623 (1.2079)  loss_classifier: 0.1692 (0.3731)  loss_box_reg: 0.0203 (0.0229)  loss_objectness: 0.0292 (0.1390)  loss_rpn_box_reg: 0.4930 (0.6730)  time: 0.2145  data: 0.0037  max mem: 5190
Epoch: [0]  [1000/2772]  eta: 0:06:27  lr: 0.000100  loss: 0.5845 (0.9700)  loss_classifier: 0.1292 (0.2737)  loss_box_reg: 0.0424 (0.0247)  loss_objectness: 0.0171 (0.0823)  loss_rpn_box_reg: 0.3856 (0.5894)  time: 0.2152  data: 0.0038  max mem: 5190
Epoch: [0]  [1500/2772]  eta: 0:04:43  lr: 0.000100  loss: 0.6308 (0.8627)  loss_classifier: 0.1192 (0.2223)  loss_box_reg: 0.0307 (0.0262)  loss_objectness: 0.0113 (0.0618)  loss_rpn_box_reg: 0.3887 (0.5523)  time: 0.2630  data: 0.0480  ma

Epoch: [4]  [1500/2772]  eta: 0:04:49  lr: 0.000010  loss: 0.4108 (0.4873)  loss_classifier: 0.0805 (0.1015)  loss_box_reg: 0.0189 (0.0292)  loss_objectness: 0.0205 (0.0158)  loss_rpn_box_reg: 0.2855 (0.3407)  time: 0.2315  data: 0.0197  max mem: 5565
Epoch: [4]  [2000/2772]  eta: 0:02:53  lr: 0.000010  loss: 0.4591 (0.4886)  loss_classifier: 0.0973 (0.1013)  loss_box_reg: 0.0233 (0.0290)  loss_objectness: 0.0225 (0.0156)  loss_rpn_box_reg: 0.2871 (0.3427)  time: 0.2149  data: 0.0040  max mem: 5565
Epoch: [4]  [2500/2772]  eta: 0:01:00  lr: 0.000010  loss: 0.4791 (0.4889)  loss_classifier: 0.0940 (0.1018)  loss_box_reg: 0.0248 (0.0292)  loss_objectness: 0.0114 (0.0156)  loss_rpn_box_reg: 0.2899 (0.3423)  time: 0.2164  data: 0.0041  max mem: 5565
Epoch: [4]  [2771/2772]  eta: 0:00:00  lr: 0.000010  loss: 0.4309 (0.4894)  loss_classifier: 0.0844 (0.1019)  loss_box_reg: 0.0267 (0.0292)  loss_objectness: 0.0124 (0.0156)  loss_rpn_box_reg: 0.2915 (0.3427)  time: 0.2169  data: 0.0043  max me

Epoch: [8]  [2500/2772]  eta: 0:00:59  lr: 0.000001  loss: 0.4245 (0.4440)  loss_classifier: 0.0925 (0.0939)  loss_box_reg: 0.0382 (0.0301)  loss_objectness: 0.0035 (0.0144)  loss_rpn_box_reg: 0.2667 (0.3055)  time: 0.2155  data: 0.0039  max mem: 5565
Epoch: [8]  [2771/2772]  eta: 0:00:00  lr: 0.000001  loss: 0.4139 (0.4426)  loss_classifier: 0.0944 (0.0936)  loss_box_reg: 0.0295 (0.0301)  loss_objectness: 0.0042 (0.0145)  loss_rpn_box_reg: 0.2739 (0.3044)  time: 0.2151  data: 0.0040  max mem: 5565
Epoch: [8] Total time: 0:10:02 (0.2172 s / it)
epoch 8 loss 0.3274722099304199 total_ats [tensor(2.4805), tensor(2.8058), tensor(3.2230), tensor(5.0569), tensor(3.9446), tensor(5.6091), tensor(4.9758), tensor(4.8037), tensor(5.4931)]
Epoch: [9]  [   0/2772]  eta: 0:19:09  lr: 0.000000  loss: 0.6829 (0.6829)  loss_classifier: 0.2133 (0.2133)  loss_box_reg: 0.0749 (0.0749)  loss_objectness: 0.0035 (0.0035)  loss_rpn_box_reg: 0.3912 (0.3912)  time: 0.4146  data: 0.1927  max mem: 5565
Epoch: [9]

In [28]:
import pickle
pickle.dump(losses, open('/scratch/nhl256/dl_project/model/object_detection_resnet18_0509_trainlosses.pickle', "wb"))
pickle.dump(total_ats, open('/scratch/nhl256/dl_project/model/object_detection_resnet18_0509_evalTotalAts.pickle', "wb"))

## Evaluate - IoU by Class

In [ ]:
def prepare_pred_results(predictions):
    pred_boxes = []
    pred_labels = []
    pred_scores = []
    for prediction in predictions:
        #print(prediction)
        if len(prediction) == 0:
            continue
        boxes = prediction["boxes"]
        boxes = reorder_coord(boxes).tolist()
        scores = prediction["scores"].tolist()
        labels = prediction["labels"].tolist()

        pred_boxes.append(boxes)
        pred_labels.append(labels)
        pred_scores.append(scores)

    return pred_boxes, pred_labels, pred_scores

def reorder_coord(boxes):
    xmin, ymin, xmax, ymax = boxes.unbind(1)
    return torch.stack((ymin, xmin, ymax, xmax), dim=1)

def prepare_gt(targets):
    gt_boxes = []
    gt_labels = []
    for target in targets:
        boxes = target['boxes']
        boxes = reorder_coord(boxes).tolist()
        labels = target["labels"].tolist()
        gt_boxes.append(boxes)
        gt_labels.append(labels)
    return gt_boxes, gt_labels

In [ ]:
# Make sure that bbox_a, bbox_b = np.array

def bbox_iou(bbox_a, bbox_b):
    #print(type(bbox_a), type(bbox_b))
    bbox_a = np.array(bbox_a)
    bbox_b = np.array(bbox_b)

    # print(type(bbox_a), type(bbox_b))
    # print(bbox_a.shape, bbox_b.shape)
    if bbox_a.shape[1] != 4 or bbox_b.shape[1] != 4:
        raise IndexError

    # top left (i.e., ymin, xmin)
    tl = np.maximum(bbox_a[:, None, :2], bbox_b[:, :2])
    # bottom right (i.e., ymax, xmax)
    br = np.minimum(bbox_a[:, None, 2:], bbox_b[:, 2:])

    # Area of intersection: (tl < br) = bool, (br-tl) = (ymax-ymin) 
    area_i = np.prod(br - tl, axis=2) * (tl < br).all(axis=2)
    area_a = np.prod(bbox_a[:, 2:] - bbox_a[:, :2], axis=1)
    area_b = np.prod(bbox_b[:, 2:] - bbox_b[:, :2], axis=1)

    return area_i / (area_a[:, None] + area_b - area_i)

In [ ]:
def cal_TP_FP_iou(pred_bbox_c, gt_bbox_c, iou_thres=0.5):
    iou_table = bbox_iou(pred_bbox_c, gt_bbox_c)
    num_pred_bboxes = iou_table.shape[0]
    num_gt_bboxes = iou_table.shape[1]
    TP = np.zeros(num_pred_bboxes)
    FP = np.zeros(num_pred_bboxes)
    # For each pred_bounding box:
      # Find the most relevant gt_bbox (i.e., the gt_bbox with max IoU)
      # If IoU < threshold, then flag it as FP
      # If IoU >= threshold, then:
        # If that gt_bbox already has already matched with another pred_bbox:
          # Flag it as FP
        # Else:
          # Flag it as TP

    # TP only happens if the pred_bbox mathes with a gt_bbox
    for i in range(num_pred_bboxes):
        gt_bbox_index = np.argmax(iou_table[i])
        best_pred_bbox_index_for_selected_gt_bbox = np.argmax(iou_table[:,gt_bbox_index])
        if iou_table[i, gt_bbox_index] > iou_thres \
            and gt_bbox_index == best_pred_bbox_index_for_selected_gt_bbox:
            TP[i] = 1
        else:
            FP[i] = 1

    TP_cum = np.sum(TP)
    FP_cum = np.sum(FP)

    if (TP_cum + FP_cum) != num_pred_bboxes:
        print("WRONG CALCULATION OF FP")
    return TP_cum, FP_cum

In [ ]:
# Test for cal_TP_FP_iou

def inspect_call_TP_FP_iou(test_images, test_targets):
    test_images = torch.stack(test_images)
    #print(test_images.shape)
    test_images = prepare_inputs(test_images)
    #print(test_images[0].shape)

    test_images = list(image.to(device) for image in test_images)
    test_targets = [{k: v.to(device) for k, v in t.items()} for t in test_targets]

    model.eval()
    predictions = model(test_images)

    pred_bboxes, pred_labels, pred_scores = prepare_pred_results(predictions)
    gt_bboxes, gt_labels = prepare_gt(test_targets)

    for pred_bbox, pred_label, pred_score, gt_bbox, gt_label in \
        zip(pred_bboxes, pred_labels, pred_scores, gt_bboxes, gt_labels):
        pred_bbox = np.array(pred_bbox)
        pred_score = np.array(pred_score)
        pred_label = np.array(pred_label)
        gt_bbox = np.array(gt_bbox)
        gt_label = np.array(gt_label)
        unique_share_classes = (np.unique(np.concatenate((pred_label, gt_label))))
        
        for c in unique_share_classes:
            pred_class_c_index = np.where(pred_label == c)[0]
            pred_bbox_c = pred_bbox[pred_class_c_index]
            gt_class_c_index = np.where(gt_label == c)[0]
            #print(gt_class_c_index)
            gt_bbox_c = gt_bbox[gt_class_c_index]
            num_gt_bboxes = len(gt_class_c_index)
            num_pred_bboxes = len(pred_class_c_index)
            print('class {} with {} gt_bboxes and {} pred_bboxes'.format(c, num_gt_bboxes, num_pred_bboxes))
            # print(num_gt_bboxes)
            # print(num_pred_bboxes)
            if num_pred_bboxes == 0:
                class_TP = 0
                class_FP = 0
                class_FN = num_gt_bboxes
            elif num_gt_bboxes == 0:
                class_TP = 0
                class_FP = num_pred_bboxes
                class_FN = 0
            else:
                class_TP, class_FP = cal_TP_FP_iou(pred_bbox_c, gt_bbox_c, iou_thres)
                class_FN = num_gt_bboxes - class_TP
                print(class_TP + class_FP == num_pred_bboxes)

In [ ]:
# for i in range(3):
#     print('Iter {}'.format(i))
#     test_images, test_targets = next(iter(test_loader))
#     inspect_call_TP_FP_iou(test_images, test_targets)

In [ ]:
def evaluate_one_batch(predictions, test_targets, res, iou_thres=0.5):

    pred_bboxes, pred_labels, pred_scores = prepare_pred_results(predictions)
    gt_bboxes, gt_labels = prepare_gt(test_targets)
    # res stores the TP_FP dict for each class
    # Each TP_FP dict stores the TP_FP for each class 
    
    batch_total_TP = 0
    batch_total_FP = 0
    batch_total_FN = 0
    batch_total_num_object = 0
    batch_res = {c: {'TP':0, 'FP': 0, 'FN': 0} for c in range(9)}

    for pred_bbox, pred_label, pred_score, gt_bbox, gt_label in \
        zip(pred_bboxes, pred_labels, pred_scores, gt_bboxes, gt_labels):

        pred_bbox = np.array(pred_bbox)
        pred_score = np.array(pred_score)
        pred_label = np.array(pred_label)
        gt_bbox = np.array(gt_bbox)
        gt_label = np.array(gt_label)
        unique_share_classes = (np.unique(np.concatenate((pred_label, gt_label))))
        
        for c in unique_share_classes:
            pred_class_c_index = np.where(pred_label == c)[0]
            pred_bbox_c = pred_bbox[pred_class_c_index]
            gt_class_c_index = np.where(gt_label == c)[0]
            #print(gt_class_c_index)
            gt_bbox_c = gt_bbox[gt_class_c_index]
            num_gt_bboxes = len(gt_class_c_index)
            num_pred_bboxes = len(pred_class_c_index)
            #print('class {} with {} gt_bboxes and {} pred_bboxes'.format(c, num_gt_bboxes, num_pred_bboxes))
            if num_pred_bboxes == 0:
                class_TP = 0
                class_FP = 0
                class_FN = num_gt_bboxes
            elif num_gt_bboxes == 0:
                class_TP = 0
                class_FP = num_pred_bboxes
                class_FN = 0
            else:
                class_TP, class_FP = cal_TP_FP_iou(pred_bbox_c, gt_bbox_c, iou_thres)
                class_FN = num_gt_bboxes - class_TP
                #print(class_TP + class_FP == num_pred_bboxes)

            batch_total_TP += class_TP
            batch_total_FP += class_FP
            batch_total_FN += class_FN
            batch_total_num_object += num_gt_bboxes

            batch_res[c]['TP'] += class_TP
            batch_res[c]['FP'] += class_FP
            batch_res[c]['FN'] += class_FN

            res[c]['TP'] += class_TP
            res[c]['FP'] += class_FP
            res[c]['FN'] += class_FN
            
    return res, batch_res, batch_total_TP, batch_total_FP, batch_total_FN, batch_total_num_object

In [ ]:
# Inspect evaluate_one_batch
def inspect_evaluate_one_batch(test_images, test_targets, final_res):
    test_images = torch.stack(test_images)
    #print(test_images.shape)
    test_images = prepare_inputs(test_images)
    #print(test_images[0].shape)

    test_images = list(image.to(device) for image in test_images)
    test_targets = [{k: v.to(device) for k, v in t.items()} for t in test_targets]

    model.eval()
    predictions = model(test_images)

    final_res, batch_res, batch_total_TP, batch_total_FP, batch_total_FN, batch_total_num_object \
    = evaluate_one_batch(predictions, test_targets, final_res, iou_thres=0.5)

    return final_res, batch_res, batch_total_TP, batch_total_FP, batch_total_FN, batch_total_num_object

In [ ]:

final_res = {c: {'TP':0, 'FP': 0, 'FN': 0} for c in range(9)}
final_TP = 0
final_FP = 0
final_FN = 0
final_num_objects = 0

# test for 2 batches
for i in range(2):
    test_images, test_targets = next(iter(test_loader))
    final_res, batch_res, batch_total_TP, batch_total_FP, batch_total_FN, batch_total_num_object \
    = inspect_evaluate_one_batch(test_images, test_targets, final_res)

    print('batch {}'.format(i))

    
    print(batch_total_TP, batch_total_FN, batch_total_num_object)
    print('cur batch res:', batch_res)
    print('final res after this batch:', final_res)

batch 0
0 20 20
cur batch res: {0: {'TP': 0, 'FP': 0, 'FN': 1}, 1: {'TP': 0, 'FP': 0, 'FN': 0}, 2: {'TP': 0, 'FP': 0, 'FN': 16}, 3: {'TP': 0, 'FP': 0, 'FN': 2}, 4: {'TP': 0, 'FP': 0, 'FN': 1}, 5: {'TP': 0, 'FP': 0, 'FN': 0}, 6: {'TP': 0, 'FP': 0, 'FN': 0}, 7: {'TP': 0, 'FP': 0, 'FN': 0}, 8: {'TP': 0, 'FP': 0, 'FN': 0}}
final res after this batch: {0: {'TP': 0, 'FP': 0, 'FN': 1}, 1: {'TP': 0, 'FP': 0, 'FN': 0}, 2: {'TP': 0, 'FP': 0, 'FN': 16}, 3: {'TP': 0, 'FP': 0, 'FN': 2}, 4: {'TP': 0, 'FP': 0, 'FN': 1}, 5: {'TP': 0, 'FP': 0, 'FN': 0}, 6: {'TP': 0, 'FP': 0, 'FN': 0}, 7: {'TP': 0, 'FP': 0, 'FN': 0}, 8: {'TP': 0, 'FP': 0, 'FN': 0}}
batch 1
0 20 20
cur batch res: {0: {'TP': 0, 'FP': 0, 'FN': 1}, 1: {'TP': 0, 'FP': 0, 'FN': 0}, 2: {'TP': 0, 'FP': 0, 'FN': 19}, 3: {'TP': 0, 'FP': 0, 'FN': 0}, 4: {'TP': 0, 'FP': 0, 'FN': 0}, 5: {'TP': 0, 'FP': 0, 'FN': 0}, 6: {'TP': 0, 'FP': 0, 'FN': 0}, 7: {'TP': 0, 'FP': 0, 'FN': 0}, 8: {'TP': 0, 'FP': 0, 'FN': 0}}
final res after this batch: {0: {'TP': 0

In [ ]:
def evaluate_one_epoch(test_loader, iou_thres=0.5):
    # Evaluate. for all data point in the evaluaton set
    final_res = {c: {'TP':0, 'FP': 0, 'FN': 0} for c in range(9)}
    final_TP = 0
    final_FP = 0
    final_FN = 0
    final_num_objects = 0

    for iter_, (test_images, test_targets) in enumerate(test_loader):
        # if iter_ % 50 == 0:
        #     print('iter', iter_)
        #print('iter', iter_)
        test_images = torch.stack(test_images)
        #print(test_images.shape)
        test_images = prepare_inputs(test_images)
        #print(test_images[0].shape)

        test_images = list(image.to(device) for image in test_images)
        test_targets = [{k: v.to(device) for k, v in t.items()} for t in test_targets]

        model.eval()
        predictions = model(test_images)

        # Evaluate for one batch
        final_res, batch_res, batch_total_TP, batch_total_FP, batch_total_FN, batch_total_num_object \
                    = evaluate_one_batch(predictions, test_targets, final_res, iou_thres=0.5)

        
        final_TP += batch_total_TP
        final_FP += batch_total_FP
        final_FN += batch_total_FN
        final_num_objects += batch_total_num_object

    return final_res, final_TP, final_FP, final_FN, final_num_objects

In [ ]:
final_res, final_TP, final_FP, final_FN, final_num_objects = evaluate_one_epoch(test_loader, iou_thres=0.5)

iter 0
iter 50
iter 100


In [ ]:
def evaluate_threst_score(TP, FP, FN):
    return (TP / (TP + FP + FN))

In [ ]:
final_res

{0: {'FN': 158, 'FP': 0, 'TP': 0},
 1: {'FN': 0, 'FP': 0, 'TP': 0},
 2: {'FN': 3074, 'FP': 0, 'TP': 0},
 3: {'FN': 390, 'FP': 0, 'TP': 0},
 4: {'FN': 27, 'FP': 0, 'TP': 0},
 5: {'FN': 0, 'FP': 0, 'TP': 0},
 6: {'FN': 20, 'FP': 0, 'TP': 0},
 7: {'FN': 0, 'FP': 0, 'TP': 0},
 8: {'FN': 0, 'FP': 0, 'TP': 0}}

## Train and Evaluate for Multiple Epochs

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001)
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10
epoch = 0
print_freq = 20

In [ ]:
def train_eval(model, train_loader, test_loader, iou_thres=0.5, num_epochs=10):
    train_losses = []
    eval_threatscores = []
    eval_final_res = []
    best_eval_ts = 0

    best_model_wts = copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        loss = train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq)
        train_losses.append(loss)
        
        final_res, final_TP, final_FP, final_FN, final_num_objects = evaluate_one_epoch(test_loader, iou_thres=0.5)

        print("epoch: {}".format(epoch))
        print(final_TP, final_FP, final_FN, final_num_objects)
        eval_final_res.append(final_res)
        eval_ts = evaluate_threst_score(final_TP, final_FP, final_FN)
        eval_threatscores.append(eval_ts)
        if epoch % 2 == 0:
            print("epoch: {} eval_ts {}".format(epoch, eval_ts))

        if eval_ts > best_eval_ts:
            best_eval_ts = eval_ts 
            best_model_wts = copy.deepcopy(model.state_dict())

    return model, best_model_wts, train_losses, eval_final_res

In [ ]:
model, best_model_wts, train_losses, eval_final_res = train_eval(model, train_loader,
                                                                 test_loader, iou_thres=0.5,
                                                                 num_epochs=10)

In [ ]:
eval_final_res

[{0: {'FN': 158, 'FP': 0, 'TP': 0},
  1: {'FN': 0, 'FP': 0, 'TP': 0},
  2: {'FN': 3073.0, 'FP': 12599.0, 'TP': 1.0},
  3: {'FN': 390, 'FP': 0, 'TP': 0},
  4: {'FN': 27, 'FP': 0, 'TP': 0},
  5: {'FN': 0, 'FP': 0, 'TP': 0},
  6: {'FN': 20, 'FP': 0, 'TP': 0},
  7: {'FN': 0, 'FP': 0, 'TP': 0},
  8: {'FN': 0, 'FP': 0, 'TP': 0}},
 {0: {'FN': 158, 'FP': 0, 'TP': 0},
  1: {'FN': 0, 'FP': 0, 'TP': 0},
  2: {'FN': 3070.0, 'FP': 846.0, 'TP': 4.0},
  3: {'FN': 390, 'FP': 0, 'TP': 0},
  4: {'FN': 27, 'FP': 0, 'TP': 0},
  5: {'FN': 0, 'FP': 0, 'TP': 0},
  6: {'FN': 20, 'FP': 0, 'TP': 0},
  7: {'FN': 0, 'FP': 0, 'TP': 0},
  8: {'FN': 0, 'FP': 0, 'TP': 0}}]

## Evaluate the trained model on the test set

## Customize Fast RCNN

In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

#### 1. Mobilenet_v2

In [ ]:
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                    aspect_ratios=((0.5, 1.0, 2.0),))
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)
model = torchvision.models.detection.faster_rcnn.FasterRCNN(backbone,
                    num_classes=21,
                    rpn_anchor_generator=anchor_generator,
                    box_roi_pool=roi_pooler)

NameError: ignored

In [ ]:
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)

#### 2. CustomVGG16

In [ ]:
def customize_VGG16():
    model = torchvision.models.vgg16(pretrained=True)
    
    features = list(model.features)[:30]
    classifier = model.classifier
    
    classifier = list(classifier)
    # delete the Linear layer
    del classifier[6]
    classifier = nn.Sequential(*classifier)

    #freeze top4 conv layer
    for layer in features[:10]:
        for p in layer.parameters():
            p.requires_grad = False
    features = nn.Sequential(*features)
        
    return features, classifier
backbone, box_head = customize_VGG16()
backbone.out_channels = 512
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                           aspect_ratios=((0.5, 1.0, 2.0),))
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)
